# БЕРЛИН ВЛАДА БКЛ-202
# Домашнее задание №1

In [1]:
import requests
from pprint import pprint
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.metrics import accuracy_score
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)
import spacy

In [2]:
session = requests.session()
sw = stopwords.words('russian')
morph = MorphAnalyzer()

nlp = spacy.load('ru_core_news_sm')

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

Притворяемся браузером, чтобы собрать данные с сайта

In [3]:
ua = UserAgent(verify_ssl=False)

In [4]:
pro_texts = []
contra_texts = []
check_texts = []

In [5]:
def chunker(text):
  ngramms = []
  dct = {'ADJF': 'ADJ', 'ADJS': 'ADJ', 'COMP': 'ADJ', 'INFN': 'VERB',
  'PRTF': 'VERB', 'PRTS': 'VERB', 'GRND': 'VERB'}

  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)

  for token in doc.tokens:
    token.lemmatize(morph_vocab)

  i = 0
  l = -2  ## На случай, если какой-то из интересующих нас частеречных тегов, являющийся вторым элементом ngramm'ы, появится раньше, чем первый элемент такой ngramm'ы
  k = -2
  m = -2

  for t in doc.tokens:

    if t.pos == 'VERB':              ## Если предыдущее слово является элементом ngramm'ы
      if i == m + 1:
        mor = morph.parse(t.text)
        for el in mor:
          if el.tag.POS in dct:
            tg = dct[el.tag.POS]
          else:
            tg = el.tag.POS
          if tg == 'VERB':
            ngramms[-1].append(el.normal_form)  ## В наших интересах записывать начальные формы, так как в таком случае от них будет больше пользы
            break
          if el == mor[-1]:             ## Если pymorphy не смог разобрать слово как нужную нам часть речи, то приходится добавлять форму, которая встретилась в тексте
            ngramms[-1].append(t.text)

    elif t.pos == 'NOUN':
      if i == k + 1:              ## Если предыдущее слово является элементом ngramm'ы
        mor = morph.parse(t.text)
        for el in mor:
          if el.tag.POS == 'NOUN':   ## Так как все теги, обозначающие прилагательные, записаны в словаре перевода тегов, можно проверять подходит ли нам этот разбор только по тем тегам, которые есть в словаре
            ngramms[-1].append(el.normal_form)
            break
          if el == mor[-1]:
            ngramms[-1].append(t.text)

    elif t.pos == 'ADJ':
      if i == l + 1:              ## Если предыдущее слово является элементом ngramm'ы
        mor = morph.parse(t.text)
        for el in mor:
          tg = ''
          if el.tag.POS in dct:
            tg = dct[el.tag.POS]
          if tg == 'ADJ':
            ngramms[-1].append(el.normal_form)
            break
          if el == mor[-1]:
            ngramms[-1].append(t.text)

    elif k == i - 1 or l == i - 1 or m == i - 1:
      ngramms = ngramms[:-1]

    if t.pos == 'ADJ':
      k = i
      ngramms.append([t.text])

    elif t.text == 'очень':
      l = i
      ngramms.append([t.text])

    elif t.text == 'не':
      m = i
      ngramms.append([t.text])

    i += 1
  return ngramms

В разделе, в котором я ищу, всего 8 страниц, поэтому я ввела такой интервал для i.
На этом сайте пользователи еще дополнительно пишут достоинства и недостатки, поэтому их я тоже буду использовать для нахождения ключевых слов.
Я решила, что положительные отзывы - это отзывы с оценками 5 и 4, а все остальные - это негативные.

In [6]:
for_check = {}
ind = 0
pos_ng = []
neg_ng = []

for i in range(1, 9):
    page_number = i

    ## Получаем доступ к странице, где представлено сколько-то игр
    url = f'https://spasibovsem.ru/igry-dlya-pk-otzyvy/?page={page_number}'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')

    ## Получаем доступ к странице с отзывами к какой-то определенной игре 
    list_games = soup.find_all('div', {'class': 'response-counter adaptive'})
    for game_ind in range(len(list_games)):
        link = list_games[game_ind].find('a').attrs['href']
        game_url = f'https://spasibovsem.ru{link}'
        game_req = session.get(game_url, headers={'User-Agent': ua.random})
        game_page = game_req.text
        game_soup = BeautifulSoup(game_page, 'html.parser')

        ## Получаем доступ к странице с определенным отзывом
        list_fb = game_soup.find_all('div', {'class': 'name'})
        for fb_ind in range(len(list_fb)):
            fb_link = list_fb[fb_ind].find('a').attrs['href']
            fb_url = f'https://spasibovsem.ru{fb_link}'
            fb_req = session.get(fb_url, headers={'User-Agent': ua.random})
            fb_page = fb_req.text
            fb_soup = BeautifulSoup(fb_page, 'html.parser')
            fb_text = fb_soup.find('div', {'class': 'text response-text description'}).text
            pro_texts.append(fb_soup.find('span', {'class': 'pro'}).text)
            contra_texts.append(fb_soup.find('span', {'class': 'contra'}).text)
            rank = fb_soup.find('abbr', {'class': 'rating'}).attrs['title']
            ng = chunker(fb_text)
            if (rank == '5' or rank == '4') and len(pro_texts) < 61:  ## Так как я добавляю еще и текст из строки "достоинства", на 30 текстов будет приходиться 60 элементов списка
                pro_texts.append(fb_text)
                pos_ng.extend(ng)
            elif rank != '5' and rank != '4' and len(contra_texts) < 61: ## То же самое, что и в случае с высокой оценкой
                contra_texts.append(fb_text)
                neg_ng.extend(ng)
            elif len(check_texts) < 11:     ## Если в списке, к которому подходит этот отзыв, уже есть 30 отзывов, и в тестовой выборке еще не собралось 10 текстов, то мы добавляем этот текст в нее
                check_texts.append(fb_text)
                if rank == '5' or rank == '4':
                    for_check[ind] = 'pro'
                    pos_ng.extend(ng)
                else:
                    for_check[ind] = 'contra'
                    neg_ng.extend(ng)
                ind += 1
            if len(check_texts) > 9 and len(pro_texts) > 60 and len(contra_texts) > 60:
                break
        if len(check_texts) > 9 and len(pro_texts) > 60 and len(contra_texts) > 60:
                break
    if len(check_texts) > 9 and len(pro_texts) > 60 and len(contra_texts) > 60:
                break

нахождение ключевых слов

In [9]:
contra_prep = []
pro_prep = []

for text in contra_texts:
    tokenized_text = [w.lower() for w in word_tokenize(text) if w.isalpha()]
    filt_words = [w for w in tokenized_text if w not in sw]
    lemmas = [morph.parse(w)[0].normal_form for w in filt_words]
    contra_prep.extend(lemmas)
    contra_count = Counter(contra_prep) # Считаем сколько раз встретилось каждое слово
    
contra_prep = list(set(contra_prep))

for text in pro_texts:
    tokenized_text = [w.lower() for w in word_tokenize(text) if w.isalpha()]
    filt_words = [w for w in tokenized_text if w not in sw]
    lemmas = [morph.parse(w)[0].normal_form for w in filt_words]
    pro_prep.extend(lemmas)
    pro_count = Counter(pro_prep)

pro_prep = list(set(pro_prep))

for w in contra_prep:
    if w in pro_prep:
        contra_prep.remove(w)   # Убираем, если слово встречается и в положительных, и в отрицательных отзывах
        pro_prep.remove(w)
    elif contra_count[w] < 10:  #Я решила брать только те слова, которые встретились 10 раз или больше
        contra_prep.remove(w)

for w in pro_prep:
    if pro_count[w] < 10:
        pro_prep.remove(w)

Определяем положительный или отрицательный перед нами отзыв и в конце проверяем accuracy того, что у нас получилось. За каждое слово, которое встречается в положительных отзывах, мы добавляем 1 к "оценке", а за слово, которое встречается в отрицательных отзывах, - добавляем -1. В итоге если "положительных" слов встретилось больше, то "оценка" отзыва больше 0, значит, он положительный, если больше "отрицательных" слов, то "оценка" меньше 0, следовательно, отзыв негативный. На случай если "оценка" остается равна 0, я добавила статус нейтральный отзыв.

## BEFORE

In [10]:
res = {}
ind = 0
exp = []
real = []

for text in check_texts:
    c = 0
    tokenized_text = [w.lower() for w in word_tokenize(text) if w.isalpha()]
    filt_words = [w for w in tokenized_text if w not in sw]
    lemmas = [morph.parse(w)[0].normal_form for w in filt_words]
    for lemma in lemmas:
        if lemma in pro_prep:
            c += 1
        elif lemma in contra_prep:
            c -= 1

    if c > 0:
        res[ind] = 'pro'
        ind += 1
    elif c < 0:
        res[ind] = 'contra'
        ind += 1
    else:
        res[ind] = 'neutral'
        ind += 1

for i, j in for_check.items():
    exp.append(j)
for i, j in res.items():
    real.append(j)
accuracy_score(real, exp)

1.0

## AFTER

In [11]:
res = {}
ind = 0
exp = []
real = []

for text in check_texts:
    c = 0
    chunks = chunker(text)
    for ch in chunks:
        if ch in pos_ng:
            c += 1
        elif ch in neg_ng:
            c -= 1
    tokenized_text = [w.lower() for w in word_tokenize(text) if w.isalpha()]
    filt_words = [w for w in tokenized_text if w not in sw]
    lemmas = [morph.parse(w)[0].normal_form for w in filt_words]
    for lemma in lemmas:
        if lemma in pro_prep:
            c += 1
        elif lemma in contra_prep:
            c -= 1

    if c > 0:
        res[ind] = 'pro'
        ind += 1
    elif c < 0:
        res[ind] = 'contra'
        ind += 1
    else:
        res[ind] = 'neutral'
        ind += 1

for i, j in for_check.items():
    exp.append(j)
for i, j in res.items():
    real.append(j)
accuracy_score(real, exp)

1.0

В моем случае accuracy не поменялась, так как она и была 1.0, но зато можно точно сказать, что качество не упало, значит, выбор именно этих ngramm не является совсем плохим.

Предложения:
1. Возможно, если использовать еще и tfidf, то результат будет более точным, хотя на моих данных это не особо можно проверить, так как у меня accuracy 1.0 
2. На самом деле, оценка 3 звезды довольно сложная для оценивания, так как в целом 3 звезды означает среднее качество, не отличное, но и не плохое, поэтому, возможно, стоит добавить отдельно еще и слова, которые встречаются в нейтральных отзывах, создать третью категорию, нейтральный отзыв
3. Еще на том сайте, на котором я брала отзывы, есть рейтинг отзыва и рейтинг пользователя, можно как-то использовать это (например, брать только те отзывы, у которых положительный рейтинг, или отзывы от пользователей с высоким рейтингом)
4. Можно сначала взять отзывы с 5 и 1 звездами и найти там какие-то ключевые слова, затем сравнить эти слова с ключевыми словами для отзывов с 4 и 2 звездами соответственно, и те слова, которые совпадут (и не попадутся среди слов для "полярных" отзывов), будут иметь больший вес в нашей функции / будут использоваться только эти слова. Если углубиться еще сильнее, то можно провести сравнение для пар 1 и 4 звезды и 5 и 2 звезды, так как можно предположить, что в отзывах с 4 звездами должна встречаться какая-то критика (тк поставили 4, а не 5), а в отзывах с 2 звездами должно встречаться что-то положительное (поставлена не 1 звезда, а 2). Тогда у нас будет еще список положительно и отрицательно окрашенных слов (но хорошо бы их сравнить с полученными до этого списками, чтобы не получилось какого-то случайного пересечения слов).